In [1]:
import numpy as np
import pandas as pd
import ast

In [2]:
# Lectura del dataset de creditos
credits = pd.read_csv('../Datasets/credits.csv', low_memory=False)
credits.head(3)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602


In [3]:
# Verificamos la cantidad de regitros nulos por columna
credits.isna().sum()

cast    0
crew    0
id      0
dtype: int64

# Desanidamos las columnas cast, crew, id y la volvemos un dataframe
* Columna cast

In [4]:
# Asegurarnos de que la columna cast sea interpretada correctamente como lista de diccionarios
credits['cast'] = credits['cast'].apply(lambda x: ast.literal_eval(x))

# Convertir cada elemento de la columna 'cast' en un DataFrame
casting = credits['cast'].apply(pd.DataFrame)

# Asignar un índice para cada fila en el DataFrame original
credits['index'] = credits.index

# Combinar todos los DataFrames individuales en uno solo
#casting_expanded = pd.concat([df.assign(index=idx) for idx, df in casting.items()])

# Añadir la columna 'id' al DataFrame desanidado y combinar todos los DataFrames individuales en uno solo
casting_expanded = pd.concat([df.assign(id=credits.loc[idx, 'id']) for idx, df in casting.items()])

In [5]:
# Eliminamos la columna 'index'
casting_expanded = casting_expanded.drop(columns=['credit_id', 'cast_id', 'profile_path', 'order'])	

*Una lista de diccionarios que representa los actores del reparto de la película. Cada diccionario contiene información detallada sobre cada miembro del reparto.*

In [6]:
casting_expanded.head()

,character,gender,id,name
0,Woody (voice),2.0,862,Tom Hanks
1,Buzz Lightyear (voice),2.0,862,Tim Allen
2,Mr. Potato Head (voice),2.0,862,Don Rickles
3,Slinky Dog (voice),2.0,862,Jim Varney
4,Rex (voice),2.0,862,Wallace Shawn


* Columna crew

In [7]:
# Asegurarnos de que la columna cast sea interpretada correctamente como lista de diccionarios
credits['crew'] = credits['crew'].apply(lambda x: ast.literal_eval(x))

# Convertir cada elemento de la columna 'crew' en un DataFrame
crew_1 = credits['crew'].apply(pd.DataFrame)

# Asignar un índice para cada fila en el DataFrame original
credits['index'] = credits.index

# Combinar todos los DataFrames individuales en uno solo
#crew_expanded = pd.concat([df.assign(index=idx) for idx, df in crew_1.items()])

# Añadir la columna 'id' al DataFrame desanidado y combinar todos los DataFrames individuales en uno solo
crew_expanded = pd.concat([df.assign(id=credits.loc[idx, 'id']) for idx, df in crew_1.items()])

In [8]:
# Eliminamos la columna 'index'
crew_expanded = crew_expanded.drop(columns=['credit_id', 'profile_path'])	

*Una lista de diccionarios que representa los miembros del equipo de producción de la película. Cada diccionario contiene información detallada sobre cada miembro del equipo.*

In [9]:
crew_expanded.head()

,department,gender,id,job,name
0,Directing,2.0,862,Director,John Lasseter
1,Writing,2.0,862,Screenplay,Joss Whedon
2,Writing,2.0,862,Screenplay,Andrew Stanton
3,Writing,2.0,862,Screenplay,Joel Cohen
4,Writing,0.0,862,Screenplay,Alec Sokolow


* Columna id

In [10]:
col_id = credits['id']
col_id.head()

0      862
1     8844
2    15602
3    31357
4    11862
Name: id, dtype: int64

* Verificamos si hay valores nulos de cada dataframe

In [11]:
print(casting_expanded.isna().sum())
print(crew_expanded.isna().sum())
print(col_id.isna().sum())

character    0
gender       0
id           0
name         0
dtype: int64
department    0
gender        0
id            0
job           0
name          0
dtype: int64
0


### Exportamos los datasets a csv

In [12]:
# casting_expanded.to_csv('../Datasets/casting_expanded.csv', index=True)
# casting_expanded.to_parquet('../Datasets/casting_expanded.parquet', index=True)
# crew_expanded.to_csv('../Datasets/crew_expanded.csv', index=True)
# crew_expanded.to_parquet('../Datasets/crew_expanded.parquet', index=True)
# col_id.to_csv('../Datasets/col_id.csv', index=True)

## Vamos a crear un dataframe para los endpoints 5 y 6

In [13]:
# Cargamos el dataset de peliculas
movies_2 = pd.read_csv('../Datasets/movies.csv', low_memory=False)

In [14]:
movies_2.head()

,budget,id_movie,popularity,release_date,revenue,runtime,title,vote_average,vote_count,release_year,return,genre_id,genre_name
0,30000000.0,862,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,1995,12.451801,16.0,Animation
1,30000000.0,862,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,1995,12.451801,35.0,Comedy
2,30000000.0,862,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,1995,12.451801,10751.0,Family
3,65000000.0,8844,17.015539,1995-12-15,262797249.0,104.0,Jumanji,6.9,2413.0,1995,4.043035,12.0,Adventure
4,65000000.0,8844,17.015539,1995-12-15,262797249.0,104.0,Jumanji,6.9,2413.0,1995,4.043035,14.0,Fantasy


In [15]:
# Asegurarse de que la columna 'id' sea del mismo tipo en todos los DataFrames
movies_2['id_movie'] = movies_2['id_movie'].astype(str)
casting_expanded['id'] = casting_expanded['id'].astype(str)
crew_expanded['id'] = crew_expanded['id'].astype(str)

In [16]:
movies_2 = movies_2.rename(columns={'id_movie':'id'})

In [17]:
# Realizamos la union de los df de movies y casting
union_mcasting = pd.merge(movies_2, casting_expanded, on='id', how='inner')

In [18]:
# Realizamos la union de los df de movies y crew
union_mcrew = pd.merge(movies_2, crew_expanded, on='id', how='inner')

In [19]:
union_mcasting.head()

,budget,id,popularity,release_date,revenue,runtime,title,vote_average,vote_count,release_year,return,genre_id,genre_name,character,gender,name
0,30000000.0,862,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,1995,12.451801,16.0,Animation,Woody (voice),2.0,Tom Hanks
1,30000000.0,862,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,1995,12.451801,16.0,Animation,Buzz Lightyear (voice),2.0,Tim Allen
2,30000000.0,862,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,1995,12.451801,16.0,Animation,Mr. Potato Head (voice),2.0,Don Rickles
3,30000000.0,862,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,1995,12.451801,16.0,Animation,Slinky Dog (voice),2.0,Jim Varney
4,30000000.0,862,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,1995,12.451801,16.0,Animation,Rex (voice),2.0,Wallace Shawn


In [20]:
union_mcrew.head()

,budget,id,popularity,release_date,revenue,runtime,title,vote_average,vote_count,release_year,return,genre_id,genre_name,department,gender,job,name
0,30000000.0,862,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,1995,12.451801,16.0,Animation,Directing,2.0,Director,John Lasseter
1,30000000.0,862,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,1995,12.451801,16.0,Animation,Writing,2.0,Screenplay,Joss Whedon
2,30000000.0,862,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,1995,12.451801,16.0,Animation,Writing,2.0,Screenplay,Andrew Stanton
3,30000000.0,862,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,1995,12.451801,16.0,Animation,Writing,2.0,Screenplay,Joel Cohen
4,30000000.0,862,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,1995,12.451801,16.0,Animation,Writing,0.0,Screenplay,Alec Sokolow


In [21]:
endpoint_casting = union_mcasting[['name', 'id', 'title', 'genre_name', 'return', 'release_date']]
endpoint_crew = union_mcrew[['name', 'job', 'id', 'title', 'return', 'budget', 'revenue', 'release_date']]

## Exportamos los dfs nuevos

In [22]:
endpoint_casting.to_parquet('../Datasets/endpoint_casting.parquet', index=False, compression='zstd')
endpoint_crew.to_parquet('../Datasets/endpoint_crew.parquet', index=False, compression='zstd')

In [23]:
union_mcasting.to_parquet('../Datasets/union_mcasting.parquet', index=False, compression='snappy')
union_mcrew.to_parquet('../Datasets/union_mcrew.parquet', index=False, compression='snappy')